# Chapter 3

In [9]:
# conda install pymysql

# Import create_engine function
from sqlalchemy import create_engine

# Create an engine to the census database
engine = create_engine('mysql+pymysql://' +
                       'student:datacamp' +
                       '@courses.csrrinzqubik.us-east-1.rds.amazonaws.com:3306/' +
                       'census')

# Print the table names
print(engine.table_names())

['census', 'state_fact']


In [10]:
# Data prep - the remote mysql works!
from sqlalchemy import select, Table, MetaData, desc, func
metadata = MetaData()
census = Table('census', metadata, autoload=True, autoload_with=engine)
connection = engine.connect()

In [11]:
# Build query to return state names by population difference from 2008 to 2000: stmt
stmt = select([census.columns.state,
               (census.columns.pop2008 - census.columns.pop2000).label('pop_change')])

# Append group by for the state: stmt
stmt = stmt.group_by(census.columns.state)

# Append order by for pop_change descendingly: stmt
stmt = stmt.order_by(desc('pop_change'))

# Return only 5 results: stmt
stmt = stmt.limit(5)

# Use connection to execute the statement and fetch all results
results = connection.execute(stmt).fetchall()

# Print the state and population change for each record
for result in results:
    print('{}:{}'.format(result.state, result.pop_change))


Texas:40137
California:35406
Florida:21954
Arizona:14377
Georgia:13357


In [12]:
# data prep - connect to employees db

# import case, cast and Float from sqlalchemy
from sqlalchemy import case, cast, Float

# Build an expression to calculate female population in 2000
female_pop2000 = func.sum(
    case([
        (census.columns.sex == 'F', census.columns.pop2000)
    ], else_=0))

# Cast an expression to calculate total population in 2000 to Float
total_pop2000 = cast(func.sum(census.columns.pop2000), Float)

# Build a query to calculate the percentage of females in 2000: stmt
stmt = select([female_pop2000 / total_pop2000 * 100])

# Execute the query and store the scalar result: percent_female
percent_female = connection.execute(stmt).scalar()

# Print the percentage
print(percent_female)


50.7455


/Users/brianosullivan/miniconda3/envs/datacamp/lib/python3.6/site-packages/sqlalchemy/dialects/mysql/base.py:1004: SAWarning: Datatype FLOAT does not support CAST on MySQL; the CAST will be skipped.
  self.dialect.type_compiler.process(cast.typeclause.type))


## Relationships

In [13]:
# data prep
state_fact = Table('state_fact', metadata, autoload=True, autoload_with=engine)

In [14]:
# Build a statement to join census and state_fact tables: stmt
stmt = select([census.columns.pop2000,
               state_fact.columns.abbreviation])

# Execute the statement and get the first result: result
result = connection.execute(stmt).first()

# Loop over the keys in the result object and print the key and value
for key in result.keys():
    print(key, getattr(result, key))

pop2000 89600
abbreviation IL


In [15]:
# Build a statement to select the census and state_fact tables: stmt
stmt = select([census, state_fact])

# Add a select_from clause that wraps a join for the census and state_fact
# tables where the census state column and state_fact name column match
stmt = stmt.select_from(
    census.join(state_fact, census.columns.state == state_fact.columns.name))

# Execute the statement and get the first result: result
result = connection.execute(stmt).first()

# Loop over the keys in the result object and print the key and value
for key in result.keys():
    print(key, getattr(result, key))


state Illinois
sex M
age 0
pop2000 89600
pop2008 95012
id 13
name Illinois
abbreviation IL
country USA
type state
sort 10
status current
occupied occupied
notes 
fips_state 17
assoc_press Ill.
standard_federal_region V
census_region 2
census_region_name Midwest
census_division 3
census_division_name East North Central
circuit_court 7


In [32]:
# Build a statement to select the state, sum of 2008 population and census
# division name: stmt
stmt = select([
    census.columns.state,
    func.sum(census.columns.pop2008),
    state_fact.columns.census_division_name
])

# Append select_from to join the census and state_fact tables by the census state and state_fact name columns
stmt = stmt.select_from(
    census.join(state_fact, census.columns.state == state_fact.columns.name)
)

# Append a group by for the state_fact name column
stmt = stmt.group_by(state_fact)

# Execute the statement and get the results: results
results = connection.execute(stmt).fetchall()

# Loop over the the results object and print each record.
for record in results:
    print(record)


('Illinois', Decimal('399087932'), 'East North Central')


In [41]:
# data prep
engine = create_engine("sqlite:///employees.sqlite")
connection = engine.connect()
metadata = MetaData()
employees = Table('employees', metadata, autoload=True, autoload_with=engine)
print(connection.execute("select * from employees").fetchall())

[(1, 'JOHNSON', 'ADMIN', 6, '12-17-1990', 18000, None, 4), (2, 'HARDING', 'MANAGER', 9, '02-02-1998', 52000, 300, 3), (3, 'TAFT', 'SALES I', 2, '01-02-1996', 25000, 500, 3), (4, 'HOOVER', 'SALES I', 2, '04-02-1990', 27000, None, 3), (5, 'LINCOLN', 'TECH', 6, '06-23-1994', 22500, 1400, 4), (6, 'GARFIELD', 'MANAGER', 9, '05-01-1993', 54000, None, 4), (7, 'POLK', 'TECH', 6, '09-22-1997', 25000, None, 4), (8, 'GRANT', 'ENGINEER', 10, '03-30-1997', 32000, None, 2), (9, 'JACKSON', 'CEO', None, '01-01-1990', 75000, None, 4), (10, 'FILLMORE', 'MANAGER', 9, '08-09-1994', 56000, None, 2), (11, 'ADAMS', 'ENGINEER', 10, '03-15-1996', 34000, None, 2), (12, 'WASHINGTON', 'ADMIN', 6, '04-16-1998', 18000, None, 4), (13, 'MONROE', 'ENGINEER', 10, '12-03-2000', 30000, None, 2), (14, 'ROOSEVELT', 'CPA', 9, '10-12-1995', 35000, None, 1)]


In [46]:
# Make an alias of the employees table: managers
managers = employees.alias()

# Build a query to select manager's and their employees names: stmt
stmt = select(
    [managers.columns.name.label('manager'),
     employees.columns.name.label('employee')]
)

# Match managers id with employees mgr: stmt
stmt = stmt.where(managers.columns.id == employees.columns.mgr)

# Order the statement by the managers name: stmt
stmt = stmt.order_by(managers.columns.name)

# Execute statement: results
results = connection.execute(stmt).fetchall()

# Print records
for record in results:
    print(record)

('FILLMORE', 'GRANT')
('FILLMORE', 'ADAMS')
('FILLMORE', 'MONROE')
('GARFIELD', 'JOHNSON')
('GARFIELD', 'LINCOLN')
('GARFIELD', 'POLK')
('GARFIELD', 'WASHINGTON')
('HARDING', 'TAFT')
('HARDING', 'HOOVER')
('JACKSON', 'HARDING')
('JACKSON', 'GARFIELD')
('JACKSON', 'FILLMORE')
('JACKSON', 'ROOSEVELT')


In [47]:
# Make an alias of the employees table: managers
managers = employees.alias()

# Build a query to select managers and counts of their employees: stmt
stmt = select([managers.columns.name, func.count(employees.columns.id)])

# Append a where clause that ensures the manager id and employee mgr are equal
stmt = stmt.where(managers.columns.id == employees.columns.mgr)

# Group by Managers Name
stmt = stmt.group_by(managers.columns.name)

# Execute statement: results
results = connection.execute(stmt).fetchall()

# print manager
for record in results:
    print(record)


('FILLMORE', 3)
('GARFIELD', 4)
('HARDING', 2)
('JACKSON', 4)


In [52]:
# data prep
engine = create_engine("sqlite:///census.sqlite")
connection = engine.connect()
metadata = MetaData()
census = Table('census', metadata, autoload=True, autoload_with=engine)
more_results = True
state_count = {}

results_proxy = connection.execute("select state from census")

In [53]:
# Start a while loop checking for more results
while more_results:
    # Fetch the first 50 results from the ResultProxy: partial_results
    partial_results = results_proxy.fetchmany(50)

    # if empty list, set more_results to False
    if partial_results == []:
        more_results = False

    # Loop over the fetched records and increment the count for the state
    for row in partial_results:
        if row.state in state_count:
            state_count[row.state] += 1
        else:
             state_count[row.state] = 1

# Close the ResultProxy, and thus the connection
results_proxy.close()

# Print the count by state
print(state_count)


{'Illinois': 172, 'New Jersey': 172, 'District of Columbia': 172, 'North Dakota': 172, 'Florida': 172, 'Maryland': 172, 'Idaho': 172, 'Massachusetts': 172, 'Oregon': 172, 'Nevada': 172, 'Michigan': 172, 'Wisconsin': 172, 'Missouri': 172, 'Washington': 172, 'North Carolina': 172, 'Arizona': 172, 'Arkansas': 172, 'Colorado': 172, 'Indiana': 172, 'Pennsylvania': 172, 'Hawaii': 172, 'Kansas': 172, 'Louisiana': 172, 'Alabama': 172, 'Minnesota': 172, 'South Dakota': 172, 'New York': 172, 'California': 172, 'Connecticut': 172, 'Ohio': 172, 'Rhode Island': 172, 'Georgia': 172, 'South Carolina': 172, 'Alaska': 172, 'Delaware': 172, 'Tennessee': 172, 'Vermont': 172, 'Montana': 172, 'Kentucky': 172, 'Utah': 172, 'Nebraska': 172, 'West Virginia': 172, 'Iowa': 172, 'Wyoming': 172, 'Maine': 172, 'New Hampshire': 172, 'Mississippi': 172, 'Oklahoma': 172, 'New Mexico': 172, 'Virginia': 172, 'Texas': 172}
